In [1]:
import { ChatMessageHistory} from 'langchain/stores/message/in_memory'
import {HumanMessage,AIMessage} from '@langchain/core/messages'

const history = new ChatMessageHistory()

In [2]:
await history.addMessage(new HumanMessage('hi'))
await history.addMessage(new AIMessage('what can I do for you?'))

In [3]:
const messages = await history.getMessages()

console.log(messages);


[
  HumanMessage {
    lc_serializable: true,
    lc_kwargs: { content: "hi", additional_kwargs: {}, response_metadata: {} },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "hi",
    name: undefined,
    additional_kwargs: {},
    response_metadata: {}
  },
  AIMessage {
    lc_serializable: true,
    lc_kwargs: {
      content: "what can I do for you?",
      additional_kwargs: {},
      response_metadata: {}
    },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "what can I do for you?",
    name: undefined,
    additional_kwargs: {},
    response_metadata: {}
  }
]


In [4]:
import {  Ollama } from "npm:/@langchain/ollama@0.1.5";
import {
  ChatPromptTemplate,
  MessagesPlaceholder,
} from "@langchain/core/prompts";

const chatModel = new Ollama({
  baseUrl: "http://localhost:11434",
  model: "qwen2.5:0.5b",
});

const prompt = ChatPromptTemplate.fromMessages([
  [
    "system",
    `You are a helpful assistant. Answer all questions to the best of your ability.
  You are talkative and provides lots of specific details from its context. 
  If the you does not know the answer to a question, it truthfully says you do not know.`,
  ],
  new MessagesPlaceholder("history_message"),
]);

const chain = prompt.pipe(chatModel);

In [5]:
import { ChatMessageHistory } from "langchain/stores/message/in_memory";
import { HumanMessage, AIMessage } from "@langchain/core/messages";

const history = new ChatMessageHistory();
await history.addMessage(new HumanMessage("hi, my name is Kai"));

const res1 = await chain.invoke({
  history_message: await history.getMessages(),
});


In [6]:
res1

"Hello Kai! It's nice to meet you. I'm here to provide information about your system and any relevant details for you. Please feel free to ask me anything you'd like to know."

In [7]:
await history.addMessage(new AIMessage(res1))
await history.addMessage(new HumanMessage('what is my name'))

In [8]:
const res2 = await chain.invoke({
  history_message: await history.getMessages(),
});


In [9]:
res2

'Your name is "Kai".'

##自动维护chat history

In [1]:
import { Ollama } from "npm:/@langchain/ollama@0.1.5";
import { ChatOpenAI } from "@langchain/openai";
import {
  ChatPromptTemplate,
  MessagesPlaceholder,
} from "@langchain/core/prompts";
import { ChatMessageHistory } from "langchain/stores/message/in_memory";
import { RunnableWithMessageHistory } from "@langchain/core/runnables";

// const chatModel = new Ollama({
//   baseUrl: "http://localhost:11434",
//   model: "qwen2.5:0.5b",
// });
const chatModel = new ChatOpenAI({
  configuration: {
    baseURL: "https://api.moonshot.cn/v1",
    apiKey: process.env.OPENAI_API_KEY,
  },
  modelName: "kimi-k2-0711-preview",
});


const prompt = ChatPromptTemplate.fromMessages([
  [
    "system",
    "You are a helpful assistant. Answer all questions to the best of your ability.",
  ],
  new MessagesPlaceholder("history_message"),
  ["human", "{input}"],
]);

const history = new ChatMessageHistory();
const chain = prompt.pipe(chatModel);

const chainWithHistory = new RunnableWithMessageHistory({
  runnable: chain,
  getMessageHistory: (_sessionId) => history,
  inputMessagesKey: "input",
  historyMessagesKey: "history_message",
});


In [2]:
const res1 = await chainWithHistory.invoke(
  { input: "hi,my name is Kai" },
  { configurable: { sessionId: "none" } }
);


In [ ]:
res1

In [3]:
const res2 = await chainWithHistory.invoke(
  { input: "我的名字叫什么" },
  { configurable: { sessionId: "none" } }
);


In [4]:
res2

AIMessage {
  lc_serializable: true,
  lc_kwargs: {
    content: "你的名字是 **Kai**。",
    additional_kwargs: { function_call: undefined, tool_calls: undefined },
    response_metadata: {}
  },
  lc_namespace: [ "langchain_core", "messages" ],
  content: "你的名字是 **Kai**。",
  name: undefined,
  additional_kwargs: { function_call: undefined, tool_calls: undefined },
  response_metadata: {
    tokenUsage: { completionTokens: 7, promptTokens: 58, totalTokens: 65 },
    finish_reason: "stop"
  }
}

In [ ]:
await history.getMessages()

自动生成chat history摘要

In [10]:
import { RunnableWithMessageHistory } from "@langchain/core/runnables";
import {
  ChatPromptTemplate,
  MessagesPlaceholder,
} from "@langchain/core/prompts";
import { ChatOpenAI } from "@langchain/openai";
import { ChatMessageHistory } from "langchain/stores/message/in_memory";
import { RunnableSequence } from "@langchain/core/runnables";
import { RunnablePassthrough } from "@langchain/core/runnables";
import { StringOutputParser } from "@langchain/core/output_parsers";
import { getBufferString } from "@langchain/core/messages";


In [ ]:


const summaryModel = new ChatOpenAI({
  configuration: {
    baseURL: "https://api.moonshot.cn/v1",
    apiKey: process.env.OPENAI_API_KEY,
  },
  modelName: "kimi-k2-0711-preview",
});

const summaryPrompt = ChatPromptTemplate.fromTemplate(`
  Progressively summarize the lines of conversation provided, adding onto the previous summary returning a new summary
  
  Current summary:
  {summary}
  
  New lines of conversation:
  {new_lines}
  
  New summary:
  `); 

  const summaryChain = RunnableSequence.from([
    summaryPrompt,
    summaryModel,
    new StringOutputParser(),
  ]);



In [12]:
const newSummary = await summaryChain.invoke({
  summary: "",
  new_lines: "I'm 18",
});


In [13]:
await summaryChain.invoke({
  summary: newSummary,
  new_lines: "I'm male",
});


"The speaker is an 18-year-old male."

In [14]:
import { RunnableWithMessageHistory } from "@langchain/core/runnables";
import {
  ChatPromptTemplate,
  MessagesPlaceholder,
} from "@langchain/core/prompts";
import { ChatOpenAI } from "@langchain/openai";
import { ChatMessageHistory } from "langchain/stores/message/in_memory";
import { RunnableSequence } from "@langchain/core/runnables";
import { RunnablePassthrough } from "@langchain/core/runnables";
import { StringOutputParser } from "@langchain/core/output_parsers";
import { getBufferString } from "@langchain/core/messages";

const summaryModel = new ChatOpenAI({
  configuration: {
    baseURL: "https://api.moonshot.cn/v1",
    apiKey: process.env.OPENAI_API_KEY,
  },
  modelName: "kimi-k2-0711-preview",
});

const summaryPrompt = ChatPromptTemplate.fromTemplate(`
  Progressively summarize the lines of conversation provided, adding onto the previous summary returning a new summary
  
  Current summary:
  {summary}
  
  New lines of conversation:
  {new_lines}
  
  New summary:
  `);

const summaryChain = RunnableSequence.from([
  summaryPrompt,
  summaryModel,
  new StringOutputParser(),
]);

const chatModel = new ChatOpenAI({
  configuration: {
    baseURL: "https://api.moonshot.cn/v1",
    apiKey: process.env.OPENAI_API_KEY,
  },
  modelName: "kimi-k2-0711-preview",
});
const chatPrompt = ChatPromptTemplate.fromMessages([
  [
    "system",
    `You are a helpful assistant. Answer all questions to the best of your ability.

  Here is the chat history summary:
  {history_summary}
  `,
  ],
  ["human", "{input}"],
]);

let summary = "";
const history = new ChatMessageHistory();

const chatChain = RunnableSequence.from([
  {
    input: new RunnablePassthrough({
      func: (input) => history.addUserMessage(input),
    }),
  },
  RunnablePassthrough.assign({
    history_summary: () => summary,
  }),
  chatPrompt,
  chatModel,
  new StringOutputParser(),
  new RunnablePassthrough({
    func: async (input) => {
      history.addAIMessage(input);
      const messages = await history.getMessages();
      const new_lines = getBufferString(messages);
      const newSummary = await summaryChain.invoke({
        summary,
        new_lines,
      });
      history.clear();
      summary = newSummary;
    },
  }),
]);


In [15]:
await chatChain.invoke('我饿了')

"先帮你解决当下饥饿！给你三套“5-10 分钟能搞定”的方案，按手边食材挑一个：\n" +
  "\n" +
  "1. 极速碳水版 – 葱香煎蛋面  \n" +
  "   一把挂面 + 2 个鸡蛋 + 葱花 + 生抽/盐/香油。煮面时顺带煎个荷包蛋，面捞起加汤或直接拌酱，撒葱花完成。\n" +
  "\n" +
  "2. 便利店极简版 – 饭团+配菜  \n" +
  "   去最近便利店买一只金枪鱼蛋黄酱饭团 + 一份即食毛豆/溏心蛋，边走边吃顶饿又抗饿。\n" +
  "\n" +
  "3. 零烹饪版 – 冰箱拼盘  \n" +
  "   冰箱里如有吐司/馒头、花生酱/辣酱、番茄/黄瓜，任意组合厚涂酱+切片蔬菜夹着吃，再来一杯牛奶。\n" +
  "\n" +
  "没有上述材料？告诉我你家里还剩啥，我帮你 1:1 现场配菜谱！"

In [16]:
await chatChain.invoke('我今天想吃方便面')

"好！既然今天特别想吃方便面，那就把“速食”做出升级版的幸福感，给你 3 条 5 分钟路线，任选或自由混搭。你只告诉我现在手头有啥——冰箱里剩下的蔬菜、肉类、罐头、酱料都算——我立刻帮你“私人定制”。\n" +
  "\n" +
  "————————\n" +
  "1. 3 级进阶版\n" +
  "经典三件套：方便面饼＋自带调料 → 煎/煮一个溏心荷包蛋 → 撒葱花＋海苔碎＋白芝麻。关键是最后 10 秒淋半勺热油（可用微波炉热 15 秒代替），滋啦一声，香气翻倍。\n" +
  "\n" +
  "2. 不用碗的韩式芝士锅\n" +
  "① 小号不粘锅直接把面饼摆上；② 倒开水没过面，中火 1 分 30 秒后把泡面自带的粉包减半量洒进去；③ 翻进两片芝士（或一小块马苏里拉）＋泡菜/午餐肉丁；④ 汤汁收浓稠时关火，用筷子搅拌拉丝。全程不进碗，锅就是碗。\n" +
  "\n" +
  "3. 5 分钟“伪拉面”\n" +
  "把面煮至 6 分熟捞出（锅里水别倒）。  \n" +
  "替换高汤：原锅水＋味噌半勺＋生抽 1 小勺＋香油几滴。  \n" +
  "直接把面回锅＋打开任意速冻丸子 4–5 粒，小青菜一把，瞬间拥有“拉面店同款”味儿。\n" +
  "\n" +
  "————————\n" +
  "如果遇到以下情况，直接告诉我，我 30 秒给你改方案：\n" +
  "- 没青菜/没蛋/没芝士  \n" +
  "- 想让方便面少油少盐  \n" +
  "- 多了一张吃剩的披萨皮/半袋速冻毛豆/任何冰箱边角料\n" +
  "\n" +
  "你现在有的配料？"